# Unsloth Environment Verification

This notebook verifies that all components are correctly installed for running Unsloth notebooks:
- GRPO (Reinforcement Learning)
- Vision fine-tuning (Ministral VL)
- fast_inference support

**Run this after rebuilding the jupyter pod to verify the environment.**

In [1]:
# Load environment variables from .env file
from dotenv import load_dotenv
import os

# Load .env from notebook directory
load_dotenv()
print(f"✓ HF_TOKEN loaded: {'Yes' if os.environ.get('HF_TOKEN') else 'No'}")

# CRITICAL: Import unsloth FIRST for proper TRL patching
import unsloth
from unsloth import FastLanguageModel, FastVisionModel
print(f"✓ unsloth: {unsloth.__version__}")

import transformers
print(f"✓ transformers: {transformers.__version__}")

import vllm
print(f"✓ vLLM: {vllm.__version__}")

import trl
print(f"✓ TRL: {trl.__version__}")

import torch
print(f"✓ PyTorch: {torch.__version__}")
print(f"✓ CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"✓ GPU: {torch.cuda.get_device_name(0)}")

✓ HF_TOKEN loaded: Yes🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.

/opt/pixi/.pixi/envs/default/lib/python3.13/site-packages/trl/__init__.py:203: UserWarning: TRL currently supports vLLM versions: 0.10.2, 0.11.0, 0.11.1, 0.11.2. You have version 0.14.0rc1.dev201+gadcf682fc.cu130 installed. We recommend installing a supported version to avoid compatibility issues.
  if is_vllm_available():

🦥 Unsloth Zoo will now patch everything to make training faster!✓ unsloth: 2025.12.10
✓ transformers: 5.0.0rc1
✓ vLLM: 0.14.0rc1.dev201+gadcf682fc
✓ TRL: 0.26.2
✓ PyTorch: 2.9.1+cu130
✓ CUDA available: True
✓ GPU: NVIDIA GeForce RTX 4080 SUPER

In [2]:
# GPU Memory Cleanup Helper
def cleanup_model(*args):
    """Clean up model(s) and free GPU memory.
    
    Usage: cleanup_model(model, tokenizer, trainer)
    Always call in finally block to ensure cleanup on exceptions.
    """
    import gc
    for obj in args:
        try:
            del obj
        except:
            pass
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    print("✓ GPU memory released")

print("✓ cleanup_model() helper defined")

✓ cleanup_model() helper defined

In [3]:
# Test imports for Reinforcement Learning notebook
print("=== GRPO/RL Imports ===")
from trl import GRPOConfig, GRPOTrainer
from datasets import Dataset
print("\u2713 All GRPO imports successful")

=== GRPO/RL Imports ===
✓ All GRPO imports successful

In [4]:
# Test imports for Vision notebook
print("=== Vision/SFT Imports ===")
from trl import SFTTrainer, SFTConfig
from unsloth.trainer import UnslothVisionDataCollator
from unsloth import is_bf16_supported
from transformers import TextStreamer
print("\u2713 All Vision imports successful")

=== Vision/SFT Imports ===
✓ All Vision imports successful

In [5]:
# Test model loading with FastLanguageModel
print("=== Testing Model Loading ===")
model, tokenizer = None, None
try:
    model, tokenizer = FastLanguageModel.from_pretrained(
        "unsloth/Ministral-3-3B-Reasoning-2512",
        max_seq_length=2048,
        load_in_4bit=True,
    )
    print(f"✓ Model loaded: {type(model).__name__}")
    print(f"✓ Tokenizer: {type(tokenizer).__name__}")
    print("✓ FastLanguageModel test PASSED")
finally:
    if model is not None:
        cleanup_model(model, tokenizer)

=== Testing Model Loading =====((====))==  Unsloth 2025.12.10: Fast Ministral3 patching. Transformers: 5.0.0rc1. vLLM: 0.14.0rc1.dev201+gadcf682fc.cu130.
   \\   /|    NVIDIA GeForce RTX 4080 SUPER. Num GPUs = 1. Max memory: 15.568 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu130. CUDA: 8.9. CUDA Toolkit: 13.0. Triton: 3.5.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!

Loading weights:   0%|          | 0/458 [00:00<?, ?it/s]

✓ Model loaded: Mistral3ForConditionalGeneration
✓ Tokenizer: PixtralProcessor
✓ FastLanguageModel test PASSED
✓ GPU memory released

In [6]:
# Test fast_inference capability
print("=== Fast Inference Check ===")
print("fast_inference=True uses vLLM as backend for 2x faster inference")
print(f"vLLM version: {vllm.__version__}")
print(f"Unsloth version: {unsloth.__version__}")

# Check if fast_inference is supported
import inspect
sig = inspect.signature(FastLanguageModel.from_pretrained)
if 'fast_inference' in sig.parameters:
    print("\u2713 fast_inference parameter available")
else:
    print("\u26a0 fast_inference parameter not found")

=== Fast Inference Check ===
fast_inference=True uses vLLM as backend for 2x faster inference
vLLM version: 0.14.0rc1.dev201+gadcf682fc
Unsloth version: 2025.12.10
✓ fast_inference parameter available

## Fast Inference Testing (vLLM Backend)

**Note:** `fast_inference=True` requires compatible vLLM/Unsloth versions. 
Current vLLM 0.14.0 has API changes that cause compatibility issues with Unsloth's LoRA manager.

The test below verifies the parameter is available. Full fast_inference testing requires:
- vLLM 0.10.2 - 0.11.2 (per TRL warning) or waiting for Unsloth update

In [7]:
# Test fast_inference=True with vLLM backend (after patch)
print("=== Fast Inference Test (vLLM Backend) ===")

from unsloth import FastLanguageModel
from vllm import SamplingParams
import torch
import time

model, tokenizer = None, None
try:
    model, tokenizer = FastLanguageModel.from_pretrained(
        "unsloth/Llama-3.2-1B-Instruct",
        max_seq_length=512,
        load_in_4bit=True,
        fast_inference=True,
        gpu_memory_utilization=0.5,
    )
    print("✓ Model loaded with fast_inference=True")

    # Test generation using vLLM's API
    FastLanguageModel.for_inference(model)
    messages = [{"role": "user", "content": "Say hello in one word."}]
    
    # Format prompt as text for vLLM backend
    prompt = tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )
    
    # Create SamplingParams object (required by vLLM 0.14)
    sampling_params = SamplingParams(
        temperature=0.1,
        max_tokens=10,
    )
    
    start = time.time()
    outputs = model.fast_generate([prompt], sampling_params=sampling_params)
    elapsed = time.time() - start

    response = outputs[0].outputs[0].text
    print(f"✓ vLLM generation completed in {elapsed:.2f}s")
    print(f"  Response: {response}")
    print("✓ Fast inference test PASSED")

except Exception as e:
    import traceback
    print(f"❌ Fast inference test FAILED: {e}")
    traceback.print_exc()

finally:
    if model is not None:
        cleanup_model(model, tokenizer)

=== Fast Inference Test (vLLM Backend) ===WARNING 01-02 21:17:56 [vllm.py:1427] Current vLLM config is not set.INFO 01-02 21:17:56 [scheduler.py:231] Chunked prefill is enabled with max_num_batched_tokens=2048.INFO 01-02 21:17:56 [vllm.py:609] Disabling NCCL for DP synchronization when using async scheduling.INFO 01-02 21:17:56 [vllm.py:614] Asynchronous scheduling is enabled.INFO 01-02 21:17:56 [vllm_utils.py:702] Unsloth: Patching vLLM v1 graph capture==((====))==  Unsloth 2025.12.10: Fast Llama patching. Transformers: 5.0.0.1. vLLM: 0.14.0rc1.dev201+gadcf682fc.cu130.
   \\   /|    NVIDIA GeForce RTX 4080 SUPER. Num GPUs = 1. Max memory: 15.568 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu130. CUDA: 8.9. CUDA Toolkit: 13.0. Triton: 3.5.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!Unsloth: vLLM loading u

/opt/pixi/.pixi/envs/default/lib/python3.13/site-packages/pydantic/type_adapter.py:605: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `enum` - serialized value may not be as expected [field_name='mode', input_value=3, input_type=int])
  return self.serializer.to_python(

WARNING 01-02 21:18:02 [arg_utils.py:1196] The global random seed is set to 0. Since VLLM_ENABLE_V1_MULTIPROCESSING is set to False, this may affect the random state of the Python process that launched vLLM.INFO 01-02 21:18:03 [model.py:517] Resolved architecture: LlamaForCausalLMINFO 01-02 21:18:03 [model.py:1688] Using max model len 512WARNING 01-02 21:18:03 [vllm.py:1427] Current vLLM config is not set.INFO 01-02 21:18:03 [scheduler.py:231] Chunked prefill is enabled with max_num_batched_tokens=2048.INFO 01-02 21:18:03 [scheduler.py:231] Chunked prefill is enabled with max_num_batched_tokens=4096.Unsloth: vLLM Bitsandbytes config using kwargs = {'load_in_8bit': False, 'load_in_4bit': True, 'bnb_4bit_compute_dtype': 'bfloat16', 'bnb_4bit_quant_storage': 'uint8', 'bnb_4bit_quant_type': 'nf4', 'bnb_4bit_use_double_quant': True, 'llm_int8_enable_fp32_cpu_offload': False, 'llm_int8_has_fp16_weight': False, 'llm_int8_skip_modules': ['lm_head', 'multi_modal_projector', 'merger', 'modality_

/opt/pixi/.pixi/envs/default/lib/python3.13/site-packages/pydantic/type_adapter.py:605: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `enum` - serialized value may not be as expected [field_name='mode', input_value=3, input_type=int])
  return self.serializer.to_python(

INFO 01-02 21:18:06 [topk_topp_sampler.py:47] Using FlashInfer for top-p & top-k sampling.INFO 01-02 21:18:06 [gpu_model_runner.py:3762] Starting to load model unsloth/llama-3.2-1b-instruct-unsloth-bnb-4bit...INFO 01-02 21:18:07 [cuda.py:315] Using AttentionBackendEnum.FLASHINFER backend.INFO 01-02 21:18:07 [bitsandbytes_loader.py:790] Loading weights with BitsAndBytes quantization. May take a while ...INFO 01-02 21:18:08 [weight_utils.py:550] No model.safetensors.index.json found in remote.

...

Unsloth: Will load unsloth/llama-3.2-1b-instruct-unsloth-bnb-4bit as a legacy tokenizer.

✓ Model loaded with fast_inference=True

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

✓ vLLM generation completed in 0.02s
  Response: Hello.
✓ Fast inference test PASSED
✓ GPU memory released

In [8]:
# Verify fast_inference parameter exists and confirm it works
print("=== Fast Inference Capability Check ===")
import inspect

# Check FastLanguageModel
sig = inspect.signature(FastLanguageModel.from_pretrained)
has_fast_inference = 'fast_inference' in sig.parameters
print(f"✓ fast_inference parameter available: {has_fast_inference}")

# Check FastVisionModel  
sig_vision = inspect.signature(FastVisionModel.from_pretrained)
has_fast_inference_vision = 'fast_inference' in sig_vision.parameters
print(f"✓ fast_inference in FastVisionModel: {has_fast_inference_vision}")

# Document current versions
print(f"\nCurrent versions:")
print(f"  vLLM: {vllm.__version__}")
print(f"  Unsloth: {unsloth.__version__}")
print(f"\n✓ fast_inference=True works with vLLM 0.14.0 (patched)")

=== Fast Inference Capability Check ===
✓ fast_inference parameter available: True
✓ fast_inference in FastVisionModel: True

Current versions:
  vLLM: 0.14.0rc1.dev201+gadcf682fc
  Unsloth: 2025.12.10

✓ fast_inference=True works with vLLM 0.14.0 (patched)

## Ministral VL (Vision) Training Verification

This section tests the complete vision model fine-tuning pipeline:
- FastVisionModel loading
- LoRA adapter configuration
- Dataset loading and formatting
- SFTTrainer training loop
- Inference after training

In [9]:
# Complete Vision Pipeline Test (self-contained)
# Tests: Model loading, LoRA, Dataset, Training (2 steps), Inference
print("=== Vision Training Pipeline Test ===")

from unsloth import FastVisionModel, is_bf16_supported
from unsloth.trainer import UnslothVisionDataCollator
from trl import SFTTrainer, SFTConfig
from datasets import load_dataset

model, tokenizer, trainer, dataset = None, None, None, None
try:
    # 1. Load model
    model, tokenizer = FastVisionModel.from_pretrained(
        "unsloth/Ministral-3-3B-Instruct-2512",
        load_in_4bit=True,
        use_gradient_checkpointing="unsloth",
    )
    print(f"✓ FastVisionModel loaded: {type(model).__name__}")

    # 2. Apply LoRA
    model = FastVisionModel.get_peft_model(
        model,
        finetune_vision_layers=True,
        finetune_language_layers=True,
        finetune_attention_modules=True,
        finetune_mlp_modules=True,
        r=16,
        lora_alpha=16,
        lora_dropout=0,
        bias="none",
        random_state=3407,
    )
    trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"✓ LoRA applied ({trainable:,} trainable params)")

    # 3. Load dataset
    dataset = load_dataset("unsloth/LaTeX_OCR", split="train[:5]")
    instruction = "Write the LaTeX representation for this image."
    
    def convert_to_conversation(sample):
        return {
            "messages": [
                {"role": "user", "content": [
                    {"type": "text", "text": instruction},
                    {"type": "image", "image": sample["image"]}
                ]},
                {"role": "assistant", "content": [
                    {"type": "text", "text": sample["text"]}
                ]}
            ]
        }
    
    converted_dataset = [convert_to_conversation(s) for s in dataset]
    print(f"✓ Dataset loaded ({len(converted_dataset)} samples)")

    # 4. Train (2 steps)
    trainer = SFTTrainer(
        model=model,
        tokenizer=tokenizer,
        data_collator=UnslothVisionDataCollator(model, tokenizer),
        train_dataset=converted_dataset,
        args=SFTConfig(
            per_device_train_batch_size=1,
            max_steps=2,
            warmup_steps=0,
            learning_rate=2e-4,
            logging_steps=1,
            fp16=not is_bf16_supported(),
            bf16=is_bf16_supported(),
            output_dir="outputs_ministral_vl_test",
            remove_unused_columns=False,
            dataset_text_field="",
            dataset_kwargs={"skip_prepare_dataset": True},
            max_seq_length=1024,
        ),
    )
    trainer_stats = trainer.train()
    print(f"✓ Training completed (loss: {trainer_stats.metrics.get('train_loss', 'N/A'):.4f})")

    # 5. Inference test
    FastVisionModel.for_inference(model)
    test_image = dataset[0]["image"]
    messages = [{"role": "user", "content": [{"type": "image"}, {"type": "text", "text": instruction}]}]
    input_text = tokenizer.apply_chat_template(messages, add_generation_prompt=True)
    inputs = tokenizer(test_image, input_text, add_special_tokens=False, return_tensors="pt").to("cuda")
    output = model.generate(**inputs, max_new_tokens=64, temperature=1.5, min_p=0.1)
    print("✓ Inference test passed")
    print("✓ Vision Training Pipeline test PASSED")

finally:
    # Always cleanup
    objs = [o for o in [model, tokenizer, trainer, dataset] if o is not None]
    if objs:
        cleanup_model(*objs)

=== Vision Training Pipeline Test =====((====))==  Unsloth 2025.12.10: Fast Ministral3 patching. Transformers: 5.0.0rc1. vLLM: 0.14.0rc1.dev201+gadcf682fc.cu130.
   \\   /|    NVIDIA GeForce RTX 4080 SUPER. Num GPUs = 1. Max memory: 15.568 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu130. CUDA: 8.9. CUDA Toolkit: 13.0. Triton: 3.5.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!

Loading weights:   0%|          | 0/458 [00:00<?, ?it/s]

✓ FastVisionModel loaded: Mistral3ForConditionalGenerationUnsloth: Making `model.base_model.model.model.vision_tower.transformer` require gradients
✓ LoRA applied (33,751,040 trainable params)

warmup_ratio is deprecated and will be removed in v5.2. Use `warmup_steps` instead.

✓ Dataset loaded (5 samples)

The model is already on multiple devices. Skipping the move to device specified in `args`.==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 5 | Num Epochs = 1 | Total steps = 2
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 2 x 1) = 2
 "-____-"     Trainable parameters = 33,751,040 of 3,882,841,088 (0.87% trained)

Unsloth: Will smartly offload gradients to save VRAM!

Step,Training Loss


✓ Training completed (loss: 3.0693)✓ Inference test passed
✓ Vision Training Pipeline test PASSED
✓ GPU memory released

## Verification Summary

If all cells above ran without errors, your environment is ready for:

1. **Ministral_3_(3B)_Reinforcement_Learning_Sudoku_Game.ipynb**
   - Uses: GRPOConfig, GRPOTrainer, FastLanguageModel
   - Status: Import verification only

2. **Ministral_3_VL_(3B)_Vision.ipynb**
   - Uses: SFTTrainer, SFTConfig, FastVisionModel, UnslothVisionDataCollator
   - Status: **Full pipeline tested** (model loading, LoRA, training, inference)

### What Was Verified
- Core imports (unsloth, transformers, vLLM, TRL, torch)
- FastLanguageModel loading (Ministral-3-3B-Reasoning)
- **fast_inference=True** works with vLLM 0.14.0 (patched)
- Full vision pipeline (load → LoRA → train → inference → cleanup)

### Design: Self-Contained Cells with Guaranteed Cleanup
Each test section uses `try/finally` blocks to ensure GPU memory is **always released**:
- Models cleaned up even on exceptions
- Cells can be run independently or re-run without kernel restart
- Single comprehensive vision test avoids OOM from loading model twice

In [ ]:
# Shutdown kernel
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(restart=False)